In [1]:
import $ivy.`org.apache.spark::spark-sql:3.1.2`
import $ivy.`org.apache.spark::spark-mllib:3.1.2`

import $ivy.$                                  

import $ivy.$                                    

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.WARN)

import org.apache.log4j.{Level, Logger}


# Spark ML для задачи оттока

Источник данных: [Kaggle](https://www.kaggle.com/sakshigoyal7/credit-card-customers)

Описания полей:

Название | Описание
:--------|:--------
CLIENTNUM | Client number. Unique identifier for the customer holding the account
Attrition_Flag | Internal event (customer activity) variable
Customer_Age | Demographic variable - Customer's Age in Years
Gender | Demographic variable - M=Male, F=Female
Dependent_count | Demographic variable - Number of dependents
Education_Level | Demographic variable - Educational Qualification of the account holder (example: high school, college graduate, etc.)
Marital_Status | Demographic variable - Married, Single, Divorced, Unknown
Income_Category | Demographic variable - Annual Income Category of the account holder (< $40K, $40K - 60K, $60K - $80K, $80K-$120K, > $120K, Unknown)
Card_Category | Product Variable - Type of Card (Blue, Silver, Gold, Platinum)
Months_on_book | Period of relationship with bank

## Загрузка и знакомство с данными

In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Spark ML Research")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/02/10 15:17:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@7290aafb
import spark.implicits._

In [4]:
val basePath = "/home/vadim/Projects/otus/SparkML"

val raw = spark
        .read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv(s"$basePath/data/BankChurners.csv")

val columns: Array[String] = raw.columns
val columnsLen: Int = columns.length
val colsToDrop: Array[String] = columns.slice(columnsLen - 2, columnsLen) :+ columns.head

val df = raw.drop(colsToDrop: _*)

df.show(5, truncate = false)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|Attrition_Flag   |Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+--------

basePath: String = "/home/vadim/Projects/otus/SparkML"
raw: DataFrame = [CLIENTNUM: int, Attrition_Flag: string ... 21 more fields]
columns: Array[String] = Array(
  "CLIENTNUM",
  "Attrition_Flag",
  "Customer_Age",
  "Gender",
  "Dependent_count",
  "Education_Level",
  "Marital_Status",
  "Income_Category",
  "Card_Category",
  "Months_on_book",
  "Total_Relationship_Count",
  "Months_Inactive_12_mon",
  "Contacts_Count_12_mon",
  "Credit_Limit",
  "Total_Revolving_Bal",
  "Avg_Open_To_Buy",
  "Total_Amt_Chng_Q4_Q1",
  "Total_Trans_Amt",
  "Total_Trans_Ct",
  "Total_Ct_Chng_Q4_Q1",
  "Avg_Utilization_Ratio",
  "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
  "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"
)
columnsLen: Int = 23
colsToDrop: Array[String] = Array(
  "Naive_Bayes_Classifier_Attrition_Flag_Card_Catego

### Определяем типы колонок

In [5]:
df.printSchema

root
 |-- Attrition_Flag: string (nullable = true)
 |-- Customer_Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Dependent_count: integer (nullable = true)
 |-- Education_Level: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income_Category: string (nullable = true)
 |-- Card_Category: string (nullable = true)
 |-- Months_on_book: integer (nullable = true)
 |-- Total_Relationship_Count: integer (nullable = true)
 |-- Months_Inactive_12_mon: integer (nullable = true)
 |-- Contacts_Count_12_mon: integer (nullable = true)
 |-- Credit_Limit: double (nullable = true)
 |-- Total_Revolving_Bal: integer (nullable = true)
 |-- Avg_Open_To_Buy: double (nullable = true)
 |-- Total_Amt_Chng_Q4_Q1: double (nullable = true)
 |-- Total_Trans_Amt: integer (nullable = true)
 |-- Total_Trans_Ct: integer (nullable = true)
 |-- Total_Ct_Chng_Q4_Q1: double (nullable = true)
 |-- Avg_Utilization_Ratio: double (nullable = true)



In [6]:
df.dtypes.foreach { dt => println(f"${dt._1}%25s\t${dt._2}") }

           Attrition_Flag	StringType
             Customer_Age	IntegerType
                   Gender	StringType
          Dependent_count	IntegerType
          Education_Level	StringType
           Marital_Status	StringType
          Income_Category	StringType
            Card_Category	StringType
           Months_on_book	IntegerType
 Total_Relationship_Count	IntegerType
   Months_Inactive_12_mon	IntegerType
    Contacts_Count_12_mon	IntegerType
             Credit_Limit	DoubleType
      Total_Revolving_Bal	IntegerType
          Avg_Open_To_Buy	DoubleType
     Total_Amt_Chng_Q4_Q1	DoubleType
          Total_Trans_Amt	IntegerType
           Total_Trans_Ct	IntegerType
      Total_Ct_Chng_Q4_Q1	DoubleType
    Avg_Utilization_Ratio	DoubleType


In [7]:
df.dtypes.groupBy(_._2).mapValues(_.length).foreach(println)

(DoubleType,5)
(StringType,6)
(IntegerType,9)


### Проверим числовые колонки

In [8]:
val numericColumns: Array[String] = df.dtypes.filter(p => p._2.equals("DoubleType") || p._2.equals("IntegerType")).map(_._1)
df.select(numericColumns.map(col): _*).summary().show

+-------+-----------------+------------------+------------------+------------------------+----------------------+---------------------+-----------------+-------------------+-----------------+--------------------+-----------------+-----------------+-------------------+---------------------+
|summary|     Customer_Age|   Dependent_count|    Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|     Credit_Limit|Total_Revolving_Bal|  Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|  Total_Trans_Amt|   Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-------+-----------------+------------------+------------------+------------------------+----------------------+---------------------+-----------------+-------------------+-----------------+--------------------+-----------------+-----------------+-------------------+---------------------+
|  count|            10127|             10127|             10127|                   10127|                 10127|              

numericColumns: Array[String] = Array(
  "Customer_Age",
  "Dependent_count",
  "Months_on_book",
  "Total_Relationship_Count",
  "Months_Inactive_12_mon",
  "Contacts_Count_12_mon",
  "Credit_Limit",
  "Total_Revolving_Bal",
  "Avg_Open_To_Buy",
  "Total_Amt_Chng_Q4_Q1",
  "Total_Trans_Amt",
  "Total_Trans_Ct",
  "Total_Ct_Chng_Q4_Q1",
  "Avg_Utilization_Ratio"
)

In [9]:
df.groupBy($"Customer_Age").count().orderBy($"Customer_Age").show(100)

+------------+-----+
|Customer_Age|count|
+------------+-----+
|          26|   78|
|          27|   32|
|          28|   29|
|          29|   56|
|          30|   70|
|          31|   91|
|          32|  106|
|          33|  127|
|          34|  146|
|          35|  184|
|          36|  221|
|          37|  260|
|          38|  303|
|          39|  333|
|          40|  361|
|          41|  379|
|          42|  426|
|          43|  473|
|          44|  500|
|          45|  486|
|          46|  490|
|          47|  479|
|          48|  472|
|          49|  495|
|          50|  452|
|          51|  398|
|          52|  376|
|          53|  387|
|          54|  307|
|          55|  279|
|          56|  262|
|          57|  223|
|          58|  157|
|          59|  157|
|          60|  127|
|          61|   93|
|          62|   93|
|          63|   65|
|          64|   43|
|          65|  101|
|          66|    2|
|          67|    4|
|          68|    2|
|          70|    1|
|          73

## Целевая колонка

In [10]:
val dft = df.withColumn("target", when($"Attrition_Flag" === "Existing Customer", 0).otherwise(1))

dft.select("Attrition_Flag", "target").show(5, truncate = false)

+-----------------+------+
|Attrition_Flag   |target|
+-----------------+------+
|Existing Customer|0     |
|Existing Customer|0     |
|Existing Customer|0     |
|Existing Customer|0     |
|Existing Customer|0     |
+-----------------+------+
only showing top 5 rows



dft: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]

### Проверка сбалансированности данных

In [11]:
dft.groupBy("target").count.show

+------+-----+
|target|count|
+------+-----+
|     1| 1627|
|     0| 8500|
+------+-----+



### Oversampling

In [12]:
val df1 = dft.filter($"target" === 1)
val df0 = dft.filter($"target" === 0)

val df1count = df1.count
val df0count = df0.count

df0count / df1count

df1: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]
df0: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]
df1count: Long = 1627L
df0count: Long = 8500L
res11_4: Long = 5L

In [13]:
val df1Over = df1
        .withColumn("dummy", explode(lit((1 to (df0count / df1count).toInt).toArray)))
        .drop("dummy")

df1Over.show(10, truncate = false)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+
|Attrition_Flag   |Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|target|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+---------

df1Over: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]

In [14]:
val data = df0.unionAll(df1Over)
data.groupBy("target").count.show

+------+-----+
|target|count|
+------+-----+
|     1| 8135|
|     0| 8500|
+------+-----+



data: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]

## Работа с признаками

### Проверим корреляции числовых признаков

In [15]:
val numericColumnsPairs = numericColumns.flatMap(f1 => numericColumns.map(f2 => (f1, f2)))

numericColumnsPairs: Array[(String, String)] = Array(
  ("Customer_Age", "Customer_Age"),
  ("Customer_Age", "Dependent_count"),
  ("Customer_Age", "Months_on_book"),
  ("Customer_Age", "Total_Relationship_Count"),
  ("Customer_Age", "Months_Inactive_12_mon"),
  ("Customer_Age", "Contacts_Count_12_mon"),
  ("Customer_Age", "Credit_Limit"),
  ("Customer_Age", "Total_Revolving_Bal"),
  ("Customer_Age", "Avg_Open_To_Buy"),
  ("Customer_Age", "Total_Amt_Chng_Q4_Q1"),
  ("Customer_Age", "Total_Trans_Amt"),
  ("Customer_Age", "Total_Trans_Ct"),
  ("Customer_Age", "Total_Ct_Chng_Q4_Q1"),
  ("Customer_Age", "Avg_Utilization_Ratio"),
  ("Dependent_count", "Customer_Age"),
  ("Dependent_count", "Dependent_count"),
  ("Dependent_count", "Months_on_book"),
  ("Dependent_count", "Total_Relationship_Count"),
  ("Dependent_count", "Months_Inactive_12_mon"),
  ("Dependent_count", "Contacts_Count_12_mon"),
  ("Dependent_count", "Credit_Limit"),
  ("Dependent_count", "Total_Revolving_Bal"),
  ("Dependen

In [16]:
for {
  x <- numericColumns
  y <- numericColumns
} yield (x, y)

res15: Array[(String, String)] = Array(
  ("Customer_Age", "Customer_Age"),
  ("Customer_Age", "Dependent_count"),
  ("Customer_Age", "Months_on_book"),
  ("Customer_Age", "Total_Relationship_Count"),
  ("Customer_Age", "Months_Inactive_12_mon"),
  ("Customer_Age", "Contacts_Count_12_mon"),
  ("Customer_Age", "Credit_Limit"),
  ("Customer_Age", "Total_Revolving_Bal"),
  ("Customer_Age", "Avg_Open_To_Buy"),
  ("Customer_Age", "Total_Amt_Chng_Q4_Q1"),
  ("Customer_Age", "Total_Trans_Amt"),
  ("Customer_Age", "Total_Trans_Ct"),
  ("Customer_Age", "Total_Ct_Chng_Q4_Q1"),
  ("Customer_Age", "Avg_Utilization_Ratio"),
  ("Dependent_count", "Customer_Age"),
  ("Dependent_count", "Dependent_count"),
  ("Dependent_count", "Months_on_book"),
  ("Dependent_count", "Total_Relationship_Count"),
  ("Dependent_count", "Months_Inactive_12_mon"),
  ("Dependent_count", "Contacts_Count_12_mon"),
  ("Dependent_count", "Credit_Limit"),
  ("Dependent_count", "Total_Revolving_Bal"),
  ("Dependent_count", "Avg

#### Вариант 1: DataFrameStatFunctions

In [17]:
val pairs = numericColumnsPairs
        .filter { p => !p._1.equals(p._2) }
        .map { p => if (p._1 < p._2) (p._1, p._2) else (p._2, p._1) }
        .distinct

val corr = pairs
        .map { p => (p._1, p._2, data.stat.corr(p._1, p._2)) }
        .filter(_._3 > 0.6)

pairs: Array[(String, String)] = Array(
  ("Customer_Age", "Dependent_count"),
  ("Customer_Age", "Months_on_book"),
  ("Customer_Age", "Total_Relationship_Count"),
  ("Customer_Age", "Months_Inactive_12_mon"),
  ("Contacts_Count_12_mon", "Customer_Age"),
  ("Credit_Limit", "Customer_Age"),
  ("Customer_Age", "Total_Revolving_Bal"),
  ("Avg_Open_To_Buy", "Customer_Age"),
  ("Customer_Age", "Total_Amt_Chng_Q4_Q1"),
  ("Customer_Age", "Total_Trans_Amt"),
  ("Customer_Age", "Total_Trans_Ct"),
  ("Customer_Age", "Total_Ct_Chng_Q4_Q1"),
  ("Avg_Utilization_Ratio", "Customer_Age"),
  ("Dependent_count", "Months_on_book"),
  ("Dependent_count", "Total_Relationship_Count"),
  ("Dependent_count", "Months_Inactive_12_mon"),
  ("Contacts_Count_12_mon", "Dependent_count"),
  ("Credit_Limit", "Dependent_count"),
  ("Dependent_count", "Total_Revolving_Bal"),
  ("Avg_Open_To_Buy", "Dependent_count"),
  ("Dependent_count", "Total_Amt_Chng_Q4_Q1"),
  ("Dependent_count", "Total_Trans_Amt"),
  ("Dependen

In [18]:
corr.sortBy(_._3).reverse.foreach { c => println(f"${c._1}%25s${c._2}%25s\t${c._3}") }

          Avg_Open_To_Buy             Credit_Limit	0.9952040726156253
          Total_Trans_Amt           Total_Trans_Ct	0.8053901681243808
             Customer_Age           Months_on_book	0.7805047706891142
    Avg_Utilization_Ratio      Total_Revolving_Bal	0.6946855441968229


#### Вариант 2: Correlation

In [19]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.sql.Row

val numericAssembler = new VectorAssembler()
  .setInputCols(numericColumns)
  .setOutputCol("features")

val numeric = numericAssembler.transform(data)
val Row(matrix: Matrix) = Correlation.corr(numeric, "features").head

22/02/10 15:17:48 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/02/10 15:17:48 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.stat.Correlation

import org.apache.spark.ml.linalg.Matrix

import org.apache.spark.sql.Row


numericAssembler: VectorAssembler = VectorAssembler: uid=vecAssembler_3419dca4111d, handleInvalid=error, numInputCols=14
numeric: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 20 more fields]
matrix: Matrix = 1.0                    -0.13575515707704905   ... (14 total)
-0.13575515707704905   1.0                    ...
0.780504770689084      -0.11728062823959522   ...
-0.026525310066417403  -0.03266417786351196   ...
0.13116552936201348    -0.010657501150598976  ...
-0.019944405497176732  -0.02566496168277401   ...
0.016657097778483475   0.06002070339266479    ...
-0.002841536593767456  0.0018723592997836101  ...
0.016929170875654043   0.05981512203616065    ...
-0.020559135165498028  -0.04794367154504479   ...
-0.06586475313340744   0.00700430540503166    ...
-0.07815083273365253   0.025513406218718206 

In [20]:
val corr2 = matrix.toArray
        .zip(numericColumnsPairs)
        .map(cnn => (cnn._2._1, cnn._2._2, cnn._1))
        .filter(_._3 < 1.0)
        .filter(_._3 > 0.6)
        .map { p => if (p._1 < p._2) (p._1, p._2, p._3) else (p._2, p._1, p._3) }
        .distinct

corr2: Array[(String, String, Double)] = Array(
  ("Customer_Age", "Months_on_book", 0.780504770689084),
  ("Avg_Open_To_Buy", "Credit_Limit", 0.9952040726156179),
  ("Avg_Utilization_Ratio", "Total_Revolving_Bal", 0.6946855441968222),
  ("Total_Trans_Amt", "Total_Trans_Ct", 0.8053901681243786)
)

In [21]:
corr2.sortBy(_._3).reverse.foreach { c => println(f"${c._1}%25s${c._2}%25s\t${c._3}") }

          Avg_Open_To_Buy             Credit_Limit	0.9952040726156179
          Total_Trans_Amt           Total_Trans_Ct	0.8053901681243786
             Customer_Age           Months_on_book	0.780504770689084
    Avg_Utilization_Ratio      Total_Revolving_Bal	0.6946855441968222


In [22]:
corr.toSet.intersect(corr2.toSet)

res21: Set[(String, String, Double)] = Set()

In [23]:
val numericColumnsFinal = numericColumns.diff(corr.map(_._2))

numericColumnsFinal: Array[String] = Array(
  "Customer_Age",
  "Dependent_count",
  "Total_Relationship_Count",
  "Months_Inactive_12_mon",
  "Contacts_Count_12_mon",
  "Avg_Open_To_Buy",
  "Total_Amt_Chng_Q4_Q1",
  "Total_Trans_Amt",
  "Total_Ct_Chng_Q4_Q1",
  "Avg_Utilization_Ratio"
)

### Категориальные признаки

#### Индексируем строковые колонки

In [24]:
import org.apache.spark.ml.feature.StringIndexer

val stringColumns = data
        .dtypes
        .filter(_._2.equals("StringType"))
        .map(_._1)
        .filter(!_.equals("Attrition_Flag"))

val stringColumnsIndexed = stringColumns.map(_ + "_Indexed")

val indexer = new StringIndexer()
        .setInputCols(stringColumns)
        .setOutputCols(stringColumnsIndexed)

val indexed = indexer.fit(data).transform(data)
indexed.show(5)

22/02/10 15:17:51 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+----------------------+-----------------------+--------------+-----------------------+---------------------+
|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|target|Marital_Status_Indexed|Income_Category_Indexed|Gender_Indexed|Education_Level_Indexed|Card_Category_Indexed|
+-----------------+------------+------+---------------+---------------+---------

import org.apache.spark.ml.feature.StringIndexer


stringColumns: Array[String] = Array(
  "Gender",
  "Education_Level",
  "Marital_Status",
  "Income_Category",
  "Card_Category"
)
stringColumnsIndexed: Array[String] = Array(
  "Gender_Indexed",
  "Education_Level_Indexed",
  "Marital_Status_Indexed",
  "Income_Category_Indexed",
  "Card_Category_Indexed"
)
indexer: StringIndexer = strIdx_490d480d6627
indexed: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 24 more fields]

#### Кодируем категориальные признаки

In [25]:
import org.apache.spark.ml.feature.OneHotEncoder

val catColumns = stringColumnsIndexed.map(_ + "_Coded")
    
val encoder = new OneHotEncoder()
        .setInputCols(stringColumnsIndexed)
        .setOutputCols(catColumns)

val encoded = encoder.fit(indexed).transform(indexed)
encoded.show(5)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+----------------------+-----------------------+--------------+-----------------------+---------------------+-----------------------------+----------------------------+--------------------+-----------------------------+---------------------------+
|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|target|Marital_Status_Indexed|Income_Category_Indexed|Gend

import org.apache.spark.ml.feature.OneHotEncoder


catColumns: Array[String] = Array(
  "Gender_Indexed_Coded",
  "Education_Level_Indexed_Coded",
  "Marital_Status_Indexed_Coded",
  "Income_Category_Indexed_Coded",
  "Card_Category_Indexed_Coded"
)
encoder: OneHotEncoder = oneHotEncoder_4f794adcd899
encoded: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 29 more fields]

### Собираем признаки в вектор

In [26]:
val featureColumns = numericColumnsFinal ++ catColumns

val assembler = new VectorAssembler()
  .setInputCols(featureColumns)
  .setOutputCol("features")

val assembled = assembler.transform(encoded)
assembled.show(5, truncate = false)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+----------------------+-----------------------+--------------+-----------------------+---------------------+-----------------------------+----------------------------+--------------------+-----------------------------+---------------------------+--------------------------------------------------------------------------------------------------------------------+
|Attrition_Flag   |Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Am

featureColumns: Array[String] = Array(
  "Customer_Age",
  "Dependent_count",
  "Total_Relationship_Count",
  "Months_Inactive_12_mon",
  "Contacts_Count_12_mon",
  "Avg_Open_To_Buy",
  "Total_Amt_Chng_Q4_Q1",
  "Total_Trans_Amt",
  "Total_Ct_Chng_Q4_Q1",
  "Avg_Utilization_Ratio",
  "Gender_Indexed_Coded",
  "Education_Level_Indexed_Coded",
  "Marital_Status_Indexed_Coded",
  "Income_Category_Indexed_Coded",
  "Card_Category_Indexed_Coded"
)
assembler: VectorAssembler = VectorAssembler: uid=vecAssembler_d2fb2b1d4913, handleInvalid=error, numInputCols=15
assembled: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 30 more fields]

### Нормализация

In [27]:
assembled.select("features").show(5, truncate = false)

+--------------------------------------------------------------------------------------------------------------------+
|features                                                                                                            |
+--------------------------------------------------------------------------------------------------------------------+
|(28,[0,1,2,3,4,5,6,7,8,9,12,17,23,25],[45.0,3.0,5.0,1.0,3.0,11914.0,1.335,1144.0,1.625,0.061,1.0,1.0,1.0,1.0])      |
|(28,[0,1,2,3,4,5,6,7,8,9,10,11,18,20,25],[49.0,5.0,6.0,1.0,2.0,7392.0,1.541,1291.0,3.714,0.105,1.0,1.0,1.0,1.0,1.0])|
|(28,[0,1,2,3,5,6,7,8,11,17,22,25],[51.0,3.0,4.0,1.0,3418.0,2.594,1887.0,2.333,1.0,1.0,1.0,1.0])                     |
|(28,[0,1,2,3,4,5,6,7,8,9,10,12,19,20,25],[40.0,4.0,3.0,4.0,1.0,796.0,1.405,1171.0,2.333,0.76,1.0,1.0,1.0,1.0,1.0])  |
|(28,[0,1,2,3,5,6,7,8,14,17,23,25],[40.0,3.0,5.0,1.0,4716.0,2.175,816.0,2.5,1.0,1.0,1.0,1.0])                        |
+-----------------------------------------------

In [28]:
import org.apache.spark.ml.feature.MinMaxScaler

val scaler = new MinMaxScaler()
  .setInputCol("features")
  .setOutputCol("scaledFeatures")

val scaled = scaler.fit(assembled).transform(assembled)

scaled.select("features", "scaledFeatures").show(5, truncate = false)

+--------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                            |scaledFeatures                                                                                                                                                                                                                      |
+--------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

import org.apache.spark.ml.feature.MinMaxScaler


scaler: MinMaxScaler = minMaxScal_2036ebc913ee
scaled: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 31 more fields]

## Feature Selection (отбор признаков)

In [29]:
import org.apache.spark.ml.feature.UnivariateFeatureSelector

val selector = new UnivariateFeatureSelector()
  .setFeatureType("continuous")
  .setLabelType("categorical")
  .setSelectionMode("percentile")
  .setSelectionThreshold(0.75)
  .setFeaturesCol("scaledFeatures")
  .setLabelCol("target")
  .setOutputCol("selectedFeatures")

val dataF = selector.fit(scaled).transform(scaled)

dataF.select("scaledFeatures", "selectedFeatures").show(5, truncate = false)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|scaledFeatures                                                                                                                                                                                                                      |selectedFeatures                                                                                                                                                                                      |
+-------------------------------------------------------------------------------------------------------------------------------------------

import org.apache.spark.ml.feature.UnivariateFeatureSelector


selector: UnivariateFeatureSelector = UnivariateFeatureSelector_049641328e27
dataF: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 32 more fields]

## Моделирование

### Обучающая и тестовая выборки

In [30]:
val tt: Array[Dataset[Row]] = dataF.randomSplit(Array(0.7, 0.3))
val training = tt(0)
val test = tt(1)

println(s"training\t${training.count}\ntest\t${test.count}")

training	11507
test	5128


tt: Array[Dataset[Row]] = Array(
  [Attrition_Flag: string, Customer_Age: int ... 32 more fields],
  [Attrition_Flag: string, Customer_Age: int ... 32 more fields]
)
training: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 32 more fields]
test: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 32 more fields]

### Логистическая регрессия

In [31]:
import org.apache.spark.ml.classification.LogisticRegression

val lr = new LogisticRegression()
        .setMaxIter(1000)
        .setRegParam(0.2)
        .setElasticNetParam(0.8)
        .setFamily("binomial")
        .setFeaturesCol("selectedFeatures")
        .setLabelCol("target")

val lrModel = lr.fit(training)

println(s"Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")

22/02/10 15:18:00 INFO OWLQN: Step Size: 0,1477
22/02/10 15:18:00 INFO OWLQN: Val and Grad Norm: 0,692728 (rel: 0,000149) 0,0203691
22/02/10 15:18:00 INFO OWLQN: Step Size: 1,000
22/02/10 15:18:00 INFO OWLQN: Val and Grad Norm: 0,692685 (rel: 6,24e-05) 0,0245072
22/02/10 15:18:00 INFO OWLQN: Step Size: 1,000
22/02/10 15:18:00 INFO OWLQN: Val and Grad Norm: 0,692646 (rel: 5,61e-05) 0,0125680
22/02/10 15:18:00 INFO OWLQN: Step Size: 1,000
22/02/10 15:18:00 INFO OWLQN: Val and Grad Norm: 0,692628 (rel: 2,65e-05) 0,0158969
22/02/10 15:18:01 INFO OWLQN: Step Size: 1,000
22/02/10 15:18:01 INFO OWLQN: Val and Grad Norm: 0,692611 (rel: 2,42e-05) 0,00936937
22/02/10 15:18:01 INFO OWLQN: Step Size: 1,000
22/02/10 15:18:01 INFO OWLQN: Val and Grad Norm: 0,692599 (rel: 1,69e-05) 0,0112808
22/02/10 15:18:01 INFO OWLQN: Step Size: 1,000
22/02/10 15:18:01 INFO OWLQN: Val and Grad Norm: 0,692586 (rel: 1,93e-05) 0,00884692
22/02/10 15:18:01 INFO OWLQN: Step Size: 4,410
22/02/10 15:18:01 INFO OWLQN: Val

Coefficients: (21,[7],[-1.4972068014146154]) Intercept: 0.21172325746669018


import org.apache.spark.ml.classification.LogisticRegression


lr: LogisticRegression = logreg_f9076e6b66d3
lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid=logreg_f9076e6b66d3, numClasses=2, numFeatures=21

### Training Summary

In [32]:
val trainingSummary = lrModel.binarySummary

println(s"accuracy: ${trainingSummary.accuracy}")
println(s"areaUnderROC: ${trainingSummary.areaUnderROC}")

accuracy: 0.6963587381593812
areaUnderROC: 0.7425160545582022


trainingSummary: org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@3bbcac22

### Проверяем модель на тестовой выборке

In [33]:
val predicted = lrModel.transform(test)

predicted.select("target", "rawPrediction", "probability", "prediction").show(10, truncate = false)

+------+--------------------------------------------+----------------------------------------+----------+
|target|rawPrediction                               |probability                             |prediction|
+------+--------------------------------------------+----------------------------------------+----------+
|0     |[-0.06861382976012356,0.06861382976012356]  |[0.48285326906287546,0.5171467309371245]|1.0       |
|0     |[0.1539112521948759,-0.1539112521948759]    |[0.5384020351886641,0.46159796481133586]|0.0       |
|0     |[-0.08594821114429924,0.08594821114429924]  |[0.478526164691816,0.521473835308184]   |1.0       |
|0     |[-0.02547943887391893,0.02547943887391893]  |[0.49363048486951866,0.5063695151304813]|1.0       |
|0     |[-0.02547943887391893,0.02547943887391893]  |[0.49363048486951866,0.5063695151304813]|1.0       |
|0     |[-0.04644194659431747,0.04644194659431747]  |[0.4883915997455215,0.5116084002544785] |1.0       |
|0     |[-0.010160683232089296,0.0101606832320

predicted: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 35 more fields]

In [34]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

val evaluator = new BinaryClassificationEvaluator().setLabelCol("target")

println(s"areaUnderROC: ${evaluator.evaluate(predicted)}\n")

areaUnderROC: 0.7515849005850296



import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator


evaluator: BinaryClassificationEvaluator = BinaryClassificationEvaluator: uid=binEval_5f3cd6484fac, metricName=areaUnderROC, numBins=1000

#### Confusion Matrix (матрица ошибок)

* True Positive (TP) - label is positive and prediction is also positive
* True Negative (TN) - label is negative and prediction is also negative
* False Positive (FP) - label is negative but prediction is positive
* False Negative (FN) - label is positive but prediction is negative

In [35]:
val tp = predicted.filter(($"target" === 1) and ($"prediction" === 1)).count
val tn = predicted.filter(($"target" === 0) and ($"prediction" === 0)).count
val fp = predicted.filter(($"target" === 0) and ($"prediction" === 1)).count
val fn = predicted.filter(($"target" === 1) and ($"prediction" === 0)).count

println(s"Confusion Matrix:\n$tp\t$fp\n$fn\t$tn\n")

Confusion Matrix:
1245	286
1281	2316



tp: Long = 1245L
tn: Long = 2316L
fp: Long = 286L
fn: Long = 1281L

#### Accuracy, Precision, Recall

* Accuracy (доля правильных ответов) = TP + TN / TP + TN + FP + FN
* Precision (точность) = TP / TP + FP
* Recall (полнота) = TP / TP + FN

In [36]:
val accuracy = (tp + tn) / (tp + tn + fp + fn).toDouble
val precision = tp / (tp + fp).toDouble
val recall = tp / (tp + fn).toDouble

println(s"Accuracy = $accuracy")
println(s"Precision = $precision")
println(s"Recall = $recall\n")

Accuracy = 0.6944227769110765
Precision = 0.8131939908556499
Recall = 0.49287410926365793



accuracy: Double = 0.6944227769110765
precision: Double = 0.8131939908556499
recall: Double = 0.49287410926365793

## Настраиваем модель (подбираем гиперпараметры)

In [37]:
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}

val paramGrid = new ParamGridBuilder()
  .addGrid(lr.regParam, Array(0.01, 0.1, 0.5))
  .addGrid(lr.fitIntercept)
  .addGrid(lr.elasticNetParam, Array(0.0, 0.5, 1.0))
  .build()

val trainValidationSplit = new TrainValidationSplit()
  .setEstimator(lr)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(paramGrid)
  .setTrainRatio(0.7)
  .setParallelism(2)

val model = trainValidationSplit.fit(dataF)

22/02/10 15:18:11 INFO StrongWolfeLineSearch: Line search t: 1.144086854510568 fval: 0.6299940135415281 rhs: 0.6927711036065174 cdd: 0.001102913592516582
22/02/10 15:18:11 INFO LBFGS: Step Size: 1,144
22/02/10 15:18:11 INFO LBFGS: Val and Grad Norm: 0,629994 (rel: 0,0906) 0,960624
22/02/10 15:18:11 INFO StrongWolfeLineSearch: Line search t: 1.0192317903987924 fval: 0.6360031357641065 rhs: 0.6927725664728059 cdd: 0.0010240167377625563
22/02/10 15:18:11 INFO LBFGS: Step Size: 1,019
22/02/10 15:18:11 INFO LBFGS: Val and Grad Norm: 0,636003 (rel: 0,0820) 0,869879
22/02/10 15:18:11 INFO LBFGS: Step Size: 1,000
22/02/10 15:18:11 INFO LBFGS: Val and Grad Norm: 0,545584 (rel: 0,134) 0,384862
22/02/10 15:18:11 INFO LBFGS: Step Size: 1,000
22/02/10 15:18:11 INFO LBFGS: Val and Grad Norm: 0,570099 (rel: 0,104) 0,239282
22/02/10 15:18:11 INFO LBFGS: Step Size: 1,000
22/02/10 15:18:11 INFO LBFGS: Val and Grad Norm: 0,518081 (rel: 0,0504) 0,0677655
22/02/10 15:18:11 INFO LBFGS: Step Size: 1,000
22/0

import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}


paramGrid: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
	logreg_f9076e6b66d3-elasticNetParam: 0.0,
	logreg_f9076e6b66d3-fitIntercept: true,
	logreg_f9076e6b66d3-regParam: 0.01
},
  {
	logreg_f9076e6b66d3-elasticNetParam: 0.0,
	logreg_f9076e6b66d3-fitIntercept: true,
	logreg_f9076e6b66d3-regParam: 0.1
},
  {
	logreg_f9076e6b66d3-elasticNetParam: 0.0,
	logreg_f9076e6b66d3-fitIntercept: true,
	logreg_f9076e6b66d3-regParam: 0.5
},
  {
	logreg_f9076e6b66d3-elasticNetParam: 0.5,
	logreg_f9076e6b66d3-fitIntercept: true,
	logreg_f9076e6b66d3-regParam: 0.01
},
  {
	logreg_f9076e6b66d3-elasticNetParam: 0.5,
	logreg_f9076e6b66d3-fitIntercept: true,
	logreg_f9076e6b66d3-regParam: 0.1
},
  {
	logreg_f9076e6b66d3-elasticNetParam: 0.5,
	logreg_f9076e6b66d3-fitIntercept: true,
	logreg_f9076e6b66d3-regParam: 0.5
},
  {
	logreg_f9076e6b66d3-elasticNetParam: 1.0,
	logreg_f9076e6b66d3-fitIntercept: true,
	logreg_f9076e6

In [38]:
model.bestModel.extractParamMap()

res37: org.apache.spark.ml.param.ParamMap = {
	logreg_f9076e6b66d3-aggregationDepth: 2,
	logreg_f9076e6b66d3-elasticNetParam: 0.0,
	logreg_f9076e6b66d3-family: binomial,
	logreg_f9076e6b66d3-featuresCol: selectedFeatures,
	logreg_f9076e6b66d3-fitIntercept: true,
	logreg_f9076e6b66d3-labelCol: target,
	logreg_f9076e6b66d3-maxBlockSizeInMB: 0.0,
	logreg_f9076e6b66d3-maxIter: 1000,
	logreg_f9076e6b66d3-predictionCol: prediction,
	logreg_f9076e6b66d3-probabilityCol: probability,
	logreg_f9076e6b66d3-rawPredictionCol: rawPrediction,
	logreg_f9076e6b66d3-regParam: 0.01,
	logreg_f9076e6b66d3-standardization: true,
	logreg_f9076e6b66d3-threshold: 0.5,
	logreg_f9076e6b66d3-tol: 1.0E-6
}

## Собираем всё вместе (Pipeline)

1. Отобрали числовые признаки: numericColumnsFinal
2. Проиндексировали строковые признаки: indexer
3. Закодировали категориальные признки: encoder
4. Собрали признаки в вектор: assembler
5. Нормализовали признаки: scaler
6. Провели отбор признаков: selector
7. Рассчитали модель: bestML 

In [39]:
import org.apache.spark.ml.Pipeline

val pipeline = new Pipeline().setStages(Array(indexer, encoder, assembler, scaler, selector, model.bestModel))

import org.apache.spark.ml.Pipeline


pipeline: Pipeline = pipeline_3df325fecde6

In [40]:
val Array(trainingData, testData) = data.randomSplit(Array(0.7, 0.3))

val pipelineModel = pipeline.fit(trainingData)

trainingData: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]
testData: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_3df325fecde6

## Сохраняем модель

In [43]:
pipelineModel.write.overwrite().save(s"$basePath/pipelineModel")